In [22]:
import pandas as pd
import re
import pickle

In [6]:
# via https://stackoverflow.com/questions/24893977/whats-the-best-way-to-regex-replace-a-string-in-python-but-keep-its-case

def replace_keep_case(word, replacement, text):
    def func(match):
        g = match.group()
        if g.islower(): return replacement.lower()
        if g.istitle(): return replacement.title()
        if g.isupper(): return replacement.upper()
        return replacement      
    return re.sub(word, func, text, flags=re.I)

In [176]:
repl = pd.read_csv("gender_options_words.txt", sep="\t")

In [177]:
repl

,F-Pro,M-Pro,Non-Pro
0,she,he,they
1,her,him,them
2,his second daughter,his second son,his second child
3,a lady,a gentleman,a gentleperson
4,this young lady,this young man,this young person
...,...,...,...
115,mamma,pappa,parent
116,mother,father,parent
117,Mrs.,Mr.,Msr.
118,Mrs. Bennet,Mr. Bennet,Msr. Bennet


In [416]:
idlookup = pickle.load(open("ids_to_names_genders.p", "rb"))

In [417]:
idlookup

{-1: {'label': 'NONE', 'orig_gender': 'O'},
 0: {'label': 'Elizabeth', 'orig_gender': 'F'},
 1: {'label': 'Darcy', 'orig_gender': 'M'},
 2: {'label': 'Jane', 'orig_gender': 'F'},
 3: {'label': 'Mrs. Bennet', 'orig_gender': 'F'},
 4: {'label': 'Mr. Bennet', 'orig_gender': 'M'},
 5: {'label': 'Lydia', 'orig_gender': 'F'},
 6: {'label': 'Bingley', 'orig_gender': 'M'},
 7: {'label': 'Wickham', 'orig_gender': 'M'},
 8: {'label': 'Miss Bingley', 'orig_gender': 'F'},
 9: {'label': 'Charlotte', 'orig_gender': 'F'},
 10: {'label': 'Gardiner', 'orig_gender': 'M'},
 11: {'label': 'Mrs. Gardiner', 'orig_gender': 'F'},
 12: {'label': 'Lady Lucas', 'orig_gender': 'F'},
 13: {'label': 'Denny', 'orig_gender': 'M'},
 14: {'label': 'Sir William Lucas', 'orig_gender': 'M'},
 15: {'label': 'Maria Lucas', 'orig_gender': 'F'},
 16: {'label': 'Georgiana', 'orig_gender': 'F'},
 17: {'label': 'Lady Catherine', 'orig_gender': 'F'},
 18: {'label': 'Kitty', 'orig_gender': 'F'},
 19: {'label': 'Mary', 'orig_gender

In [180]:
name_variants = {
    0: ["Elizabeth", "Lizzy", "Eliza", "Miss Bennet", "Elizabeth Bennet", "Miss Elizabeth Bennet", 
        "Miss Eliza", "Miss Elizabeth", "Miss Lizzy"],
    1: ["Darcy", "Mr. Darcy", "Fitzwilliam Darcy"],
    2: ["Jane", "Miss Jane Bennet", "Miss Bennet", "Jane Bennet"],
    6: ["Bingley", "Charles Bingley", "Mr. Bingley"],
    3: ["Mrs. Bennet"],
    4: ["Mr. Bennet"],
    5: ["Lydia"],
    18: ["Kitty", "Catherine"],
    19: ["Mary"],
    24: ["The Bennets", "daughters", "girls", "ladies", "sisters"]
}

## Would Do This Via Web Form or Something

Also, it's confusing not to fix Miss Bingely - she collides with the new one here

In [410]:
name_replacements = {
    0: {"Elizabeth": "Edward", "LIZZY": "EDDIE", "Lizzy": "Eddie", "Eliza": "Ed", "Miss Bennet": "Mr. Bennet", "Elizabeth Bennet": "Edward Bennet", "Miss Eliza": "Mr. Ed", "Miss Elizabeth": "Mr. Edward", "Miss Lizzy": "Mr. Eddie"},
    1: {"Darcy": "Darcy", "Mr. Darcy": "Miss Darcy", "Fitzwilliam Darcy": "Philomena Darcy"},
    2: {"Jane": "John", "Jane Bennet": "John Bennet", "Miss Bennet": "Mr. Bennet"},
    6: {"Bingley": "Bingley", "Charles": "Charlotte", "Mr. Bingley": "Miss Bingley", "Charles Bingley": "Charlotte Bingley"},
    3: {"Mrs. Bennet": "Mr. Bennet"},
    4: {"Mr. Bennet": "Mrs. Bennet"},
    5: {"Lydia": "Luke"},
    18: {"Kitty": "Connie", "Catherine": "Constance"},
    19: {"Mary": "Morris"},
    24: {"daughters": "sons", "girls": "boys", "ladies": "gentlemen", "sisters": "brothers"}
}

swaps = [0, 1, 2, 3, 4, 6, 5, 18, 19, 24]

### Code For It

In [320]:
def get_best_repl(inputtext, inputcol, outputcol):
    hits = repl[[bool(re.findall(r"\b" + x + r"\b", r"\b" + inputtext + r"\b")) for x in repl[inputcol]]]
    hits['len'] = hits[inputcol].apply(lambda x: len(x))
    if len(hits) > 0:
        best = hits.sort_values(by="len", ascending=False).iloc[0]
        result = replace_keep_case(best[inputcol], best[outputcol], inputtext)
        #print("got best match", result)
    else:
        result = inputtext
    return result

def check_substring(id, name):
    for key, val in name_replacements[id].items():
        if key in name:
            return replace_keep_case(key, val, name)
    return None

def try_name(id, string):
    if string in name_replacements[id]:
        return name_replacements[id][string]
    else:
        return check_substring(id, string)

def get_exact(repl, col, match, returncol):
    res = repl[repl[col]==match]
    if len(res) > 0:
        #print("got exact")
        return res[returncol].values[0]
    else:
        return None

In [247]:
def do_replace(entid, oldstring):
    if entid in swaps:
        new = None
        if newlookup[entid]['orig_gender'] == "M":
            sourcepron = 'M-Pro'
            newpron = "F-Pro"
        if newlookup[entid]['orig_gender'] == "F":
            newpron = "M-Pro"
            sourcepron = "F-Pro"
        new = try_name(entid, oldstring)
        if not new:
            new = get_exact(repl, sourcepron, oldstring, newpron)
        if not new:
            new = get_best_repl(oldstring, sourcepron, newpron)
        if new:
            return new
        else:
            return oldstring
    return oldstring

In [375]:
def special_case_check(i, df, start, oldstring, newstring):
    # fix a possessive his / him problem in compound nouns
    if oldstring == "her":
        if (df.loc[i+1]['start'] == start) and (newstring == "him"):
            print("special case match, returning his", oldstring, newstring)
            return "his"
        else:
            return newstring
    else:
        return newstring

In [326]:
!mkdir drafts

mkdir: drafts: File exists


In [290]:
!mkdir final

In [413]:
ann_sourcepath = "cl-coref-annotator/newchaps"
text_path = "cl-coref-annotator/chaps"

In [414]:
def write_chapter_draft(num):
    mergedf = pd.read_csv(f"{ann_sourcepath}/merged_data_{num}.tsv", sep="\t")
    mergedf['len'] = mergedf['string'].apply(lambda x: len(x))
    mergedf.sort_values(by=["start", "len"], ascending=True, inplace=True)
    filename = f"{text_path}/austen_chap{num}_brat.txt"
    text = None
    with open(filename) as handle:
        text = handle.read()
    with open(f"./drafts/chapter_{str(num)}.txt", "w") as handle:
        current = 0
        for i, row in mergedf.iterrows():
            start = row['start']
            ent = row['id']
            oldstring = row['string']
            end = row['end']
            if row['start'] > current:
                handle.write(text[current: start-1] + " ")
                replacement = do_replace(ent, oldstring)
                replacement = special_case_check(i, mergedf, start, oldstring, replacement)
                print("new replacement", replacement)
                handle.write(replacement + " ")
                if replacement != oldstring:
                    replaced = True
                else:
                    replaced = False
                lastlen = end - start
                current = row['end'] + 1
            else:
                print("overlapping entity", oldstring, oldstring[lastlen:])
                if replaced:
                    print("replaced is true", oldstring[lastlen:])
                    oldstring = oldstring[lastlen:]
                replacement = do_replace(ent, oldstring)
                print("after replacement2", replacement)
                handle.write(replacement + " ")
                lastlen = end - start
                current = row['end'] + 1
        handle.write(text[current:])

In [396]:
def clean_draft(text):
    text = text.replace("`` ", '"')
    text = text.replace(" ''", '"')
    text = text.replace(" _ ", " ")
    text = text.replace("_ ", "")
    text = text.replace(" ,", ",")
    text = text.replace(" .", ".")
    text = text.replace(" ,", ",")
    text = text.replace(" !", "!")
    text = text.replace(" ?", "?")
    text = text.replace(" ;", ";")
    text = text.replace(" :", ":")
    text = text.replace("  ", " ")
    text = text.replace(" n't", "n't")
    text = text.replace(" 'm", "'m")
    text = text.replace(" 's", "'s")
    text = text.replace("-LRB-", " ")
    text = text.replace("-RRB-", " ")
    text = text.replace("  ", " ")
    text = re.sub(r"(Chapter [0-9]+) ", r"\1\n\n", text)
    return text

In [411]:
def write_clean_chapter(i):
    with open(f"./drafts/chapter_{str(i)}.txt", "r") as handle:
        text = handle.read()
        text = clean_draft(text)
    with open(f"./final/chapter_{str(i)}.txt", "w") as handle:
        handle.write(text)

In [412]:
for i in range(0, 60):
    write_chapter_draft(i)
    write_clean_chapter(i)

/Users/cherny/miniconda3/envs/pandasnlp/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


new replacement a single woman in possession of a good fortune
new replacement a husband
new replacement such a woman
new replacement her
new replacement a neighbourhood
new replacement the surrounding families
new replacement she
new replacement their sons
new replacement My dear
overlapping entity Mr. Bennet net
after replacement2 Mrs. Bennet
new replacement her
overlapping entity his lady  lady
replaced is true  lady
after replacement2  gentleman
new replacement her
new replacement Netherfield Park
new replacement Mrs. Bennet
new replacement she
new replacement he
new replacement Mrs. Long
new replacement here
new replacement she
new replacement Mrs. Bennet
new replacement her
overlapping entity his wife  wife
replaced is true  wife
after replacement2  husband
new replacement Mrs. Long
new replacement Netherfield
new replacement a young woman of large fortune
new replacement the north of England
new replacement she
new replacement a chaise
new replacement the place
new replacement i

new replacement the village
new replacement the principal inhabitants
new replacement Mrs. Bennet
new replacement she
new replacement she
new replacement She
new replacement her
overlapping entity his wife  wife
replaced is true  wife
after replacement2  husband
new replacement the stranger
new replacement she
new replacement she
new replacement Mrs. Bennet
new replacement he
new replacement the room
new replacement there
new replacement John
new replacement Everybody
new replacement he
new replacement Miss Bingley
new replacement him
new replacement him
new replacement she
new replacement him
new replacement he
new replacement the room
new replacement she
new replacement she
new replacement Miss Lucas
new replacement her
new replacement her
new replacement she
new replacement her
new replacement nobody
new replacement she
new replacement John
new replacement he
new replacement she
new replacement he
new replacement him
new replacement she
new replacement Miss King
new replacement Mari

new replacement him
new replacement her
new replacement she
new replacement a simpleton
new replacement Ed
new replacement she
new replacement John
new replacement a man
new replacement a woman
new replacement she
new replacement she
new replacement she
new replacement him
new replacement Bingley
new replacement John
new replacement John
new replacement he
new replacement her
new replacement he
new replacement her
new replacement he
new replacement Edward
new replacement a rich wife
new replacement any wife
new replacement John
new replacement he
new replacement he
new replacement him
new replacement He
new replacement her
new replacement He
new replacement her
new replacement Meryton
new replacement he
new replacement her
new replacement her
overlapping entity his own house  own house
replaced is true  own house
after replacement2  own house
new replacement her
new replacement him
new replacement her
new replacement he
new replacement her
new replacement he
new replacement she
new rep

new replacement my dear
new replacement Mrs. Bennet
new replacement Edward
new replacement your
overlapping entity your daughter  daughter
after replacement2 your son
new replacement he
new replacement Miss Bingley
new replacement your
new replacement him
new replacement People
new replacement He
new replacement he
new replacement there
new replacement him
new replacement the carriage
new replacement Edward
new replacement him
new replacement the carriage
new replacement he
new replacement no horseman
new replacement him
new replacement He
new replacement him
special case match, returning his her him
new replacement his
overlapping entity her mother  mother
replaced is true  mother
after replacement2  father
new replacement this dirt
new replacement You
new replacement there
new replacement John
new replacement Eddie
special case match, returning his her him
new replacement his
overlapping entity her father  father
replaced is true  father
after replacement2  mother
new replacement you

...

new replacement she
new replacement Edward
new replacement himself
new replacement Colonel Fitzwilliam
new replacement Your cousin
new replacement a person
new replacement a part of the world where I had hoped to pass myself
overlapping entity the world 
after replacement2 the world
new replacement Miss Darcy
new replacement Hertfordshire
new replacement she
new replacement her
new replacement Colonel Fitzwilliam
new replacement she
new replacement strangers
new replacement her
new replacement Hertfordshire
new replacement she
new replacement She
new replacement ladies
new replacement one young gentleman
new replacement a partner
new replacement Miss Darcy
new replacement any gentleman
new replacement my own party
new replacement nobody
new replacement a ball-room
new replacement Colonel Fitzwilliam
new replacement Darcy
new replacement strangers
new replacement your cousin
new replacement Edward
new replacement Colonel Fitzwilliam
new replacement her
new replacement a woman of sense a

new replacement Edward
new replacement him
new replacement the park
new replacement Miss Darcy
new replacement He
new replacement her
new replacement no one else
new replacement her
new replacement a favourite haunt of hers
new replacement she
new replacement him
new replacement She
new replacement he
new replacement himself
new replacement her
new replacement she
new replacement him
new replacement Hunsford
new replacement him
new replacement him
new replacement Mrs. Collins
new replacement Rosings
new replacement him
new replacement the house
new replacement she
new replacement he
new replacement Kent
new replacement he
new replacement there _
new replacement Her
new replacement she
new replacement Colonel Fitzwilliam
new replacement her
new replacement He
new replacement she
new replacement she
new replacement him
new replacement he
new replacement himself
new replacement the pales
new replacement the Parsonage
new replacement He
new replacement he
new replacement John
new replaceme

new replacement Edward
new replacement him
new replacement He
new replacement he
new replacement himself
new replacement He
new replacement him
new replacement Miss Darcy
new replacement there
new replacement him
new replacement the park
new replacement he
new replacement the lane
new replacement the turnpike-road
new replacement The park paling
new replacement he
new replacement the lane
new replacement he
new replacement the park
new replacement he
new replacement Kent
new replacement the country
new replacement He
new replacement him
new replacement he
new replacement a lady
new replacement the sort of grove which edged the park
overlapping entity the park 
after replacement2 the park
new replacement she
new replacement Miss Darcy
new replacement he
new replacement the person who advanced
new replacement him
new replacement him
new replacement He
new replacement himself
new replacement Miss Darcy
new replacement he
new replacement She
new replacement he
new replacement the grove
new

new replacement he
new replacement she
new replacement she
new replacement he
new replacement her
new replacement Netherfield
new replacement He
new replacement him
new replacement him
new replacement Her
special case match, returning his her him
new replacement his
overlapping entity her sister  sister
replaced is true  sister
after replacement2  brother
new replacement he
new replacement her
new replacement him
new replacement her
new replacement She
new replacement she
new replacement him
new replacement her
new replacement her
new replacement Mr. Wickham
new replacement he
new replacement his
new replacement his
new replacement himself
new replacement him
new replacement him
new replacement He
new replacement he
new replacement he
new replacement he
new replacement he
new replacement himself
new replacement he
new replacement he
new replacement Wickham
new replacement himself
new replacement her
new replacement the Pemberley family
new replacement he
new replacement himself
new rep

KeyError: 571